In [ ]:
# to disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [1]:
import os
os.chdir('T:/Projects/LSTM-forecast/src')
import featureEng as fe
import processStocks as ps
import lstm
import getStocks
import visualize
import predicts

Using TensorFlow backend.


In [2]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5252221894472859956
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6772842168
locality {
  bus_id: 1
}
incarnation: 5044062486282099467
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


# User parameters

In [93]:
####TODO: reshaping so we can plot various n_in, n_out
# model seems to work, but cant redim for plot
ticker = 'unh'

n_in = 10
n_out = 5
target = 'Close'

In [94]:
import imp
imp.reload(getStocks)

<module 'getStocks' from 'T:\\Projects\\LSTM-forecast\\src\\getStocks.py'>

In [95]:
# collect a bunch of data from the market, save it to directory
# getStocks.get_mkt_data(source='yahoo')

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

In [149]:
# load dataset
#dataset = getStocks.get_single(ticker=ticker, save=True)
# dataset = getStocks.load_single(ticker)
dataset = getStocks.join_tgt_spt(target_ticker='UNH', number_spt=25)
dataset.reset_index(level=0, inplace=True)

#dataset.rename(columns={'Adj Close':'AdjCls'}, inplace=True)

In [150]:
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume,MACOpen,MACHigh,MACLow,...,TAPLow,TAPClose,TAPAdj Close,TAPVolume,CNCOpen,CNCHigh,CNCLow,CNCClose,CNCAdj Close,CNCVolume
0,1984-10-17,0.144531,0.148438,0.144531,0.144531,0.126899,9868800,NaN,NaN,NaN,...,7.4375,7.4375,3.989465,12600,NaN,NaN,NaN,NaN,NaN,NaN
1,1984-10-18,0.148438,0.156250,0.148438,0.148438,0.130329,5324800,NaN,NaN,NaN,...,7.6875,7.6875,4.123566,50400,NaN,NaN,NaN,NaN,NaN,NaN
2,1984-10-19,0.148438,0.148438,0.144531,0.144531,0.126899,3043200,NaN,NaN,NaN,...,7.5625,7.5625,4.056516,56000,NaN,NaN,NaN,NaN,NaN,NaN
3,1984-10-22,0.144531,0.148438,0.144531,0.144531,0.126899,2326400,NaN,NaN,NaN,...,7.5000,7.5000,4.022991,36600,NaN,NaN,NaN,NaN,NaN,NaN
4,1984-10-23,0.144531,0.144531,0.140625,0.140625,0.123469,787200,NaN,NaN,NaN,...,7.3750,7.3750,3.955941,54800,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Engineering
* Add day of week feature
* Move target feature to end of dataframe

In [151]:
## Generate new features
dataset = fe.derivative(dataset, drop_na = True)
dataset = fe.weekDay(dataset)
features = list(dataset.columns)
# move target to position [-1]
features.remove(target)
features.append(target)
features=list(features)
dataset=dataset[features]

In [152]:
# Plot the features
# visualize.plot_features(dataset)

# Format as Supervised Learning Problem

In [153]:
# frame as supervised learning
# this will be for scaling the data to the window scaled_in to scaled_out
reframed=ps.series_to_supervised(dataset, n_in=n_in, n_out=n_out, 
                                   features=features)

starting Series to supverised conversion...


In [154]:
# drop all but the 'target' from the predictor set
# this might be able to take an array for multi-output
reframed=ps.frame_targets(reframed, features, n_out,target=target)

#scaler = MinMaxScaler(feature_range=(-1, 1))
#scaled = scaler.fit_transform(reframed)
# pass the feature in the sequence that we dont want scaled

# Scaling - several methods
* yet to be determined if scaling improves performance

In [ ]:
#reframed=reframed.iloc[:10,:] #test/debug
#dataset=reframed
scaled=ps.scale_sequence(reframed, features, 
                         scaleTarget=True, target=target)

In [155]:
# or skip the scaling
scaled=reframed

In [156]:
#getStocks.saveScaled(scaled, n_in, n_out, ticker)
# load scaled 

#scaled=pd.read_csv('../data/astc/astc_29092017_scaled_30_30.csv')

# Cross Validation Setup

In [21]:
# put in order of time (t-n_in should be on left of df, with t+ on the right)

# split into train, validation, test
train, validation, test = lstm.tscv(scaled, train=0.7, validation=0.25)

In [33]:
# split into input and outputs
# the last n columns are the output variable

train_X, train_y = train[:, :-n_out], train[:, -n_out:]
X_validation, Y_validation = validation[:, :-n_out], validation[:, -n_out:]
test_X, test_y = test[:, :-n_out], test[:, -n_out:]

# *** Test Reshaping Method

In [173]:
# test dimension modification
import pandas as pd
df=pd.read_csv('testDim.csv')
features_test=list(df.columns)
df.head()

,Var0,Var1,Var2,Var3,Var4
0,100,101,102,103,104
1,200,201,202,203,204
2,300,301,302,303,304
3,400,401,402,403,404
4,500,501,502,503,504


In [174]:
n_in=2
n_out=1
tgt_tst='Var4'
df=ps.series_to_supervised(df, 
                           n_in=n_in, 
                           n_out=n_out, 
                           features=features_test)

starting Series to supverised conversion...


In [175]:
test_frame=ps.frame_targets(df, 
                          features=features_test, 
                          n_out=n_out,
                          target=tgt_tst)

In [188]:
X = test_frame.values[:, :-n_out]

In [192]:
[print(x) for x in X]

[ 100.  101.  102.  103.  104.  200.  201.  202.  203.  204.]
[ 200.  201.  202.  203.  204.  300.  301.  302.  303.  304.]
[ 300.  301.  302.  303.  304.  400.  401.  402.  403.  404.]
[ 400.  401.  402.  403.  404.  500.  501.  502.  503.  504.]
[ 500.  501.  502.  503.  504.  600.  601.  602.  603.  604.]
[ 600.  601.  602.  603.  604.  700.  701.  702.  703.  704.]
[ 700.  701.  702.  703.  704.  800.  801.  802.  803.  804.]
[ 800.  801.  802.  803.  804.  900.  901.  902.  903.  904.]
[  900.   901.   902.   903.   904.  1000.  1001.  1002.  1003.  1004.]
[ 1000.  1001.  1002.  1003.  1004.  1100.  1101.  1102.  1103.  1104.]
[ 1100.  1101.  1102.  1103.  1104.  1200.  1201.  1202.  1203.  1204.]
[ 1200.  1201.  1202.  1203.  1204.  1300.  1301.  1302.  1303.  1304.]
[ 1300.  1301.  1302.  1303.  1304.  1400.  1401.  1402.  1403.  1404.]
[ 1400.  1401.  1402.  1403.  1404.  1500.  1501.  1502.  1503.  1504.]
[ 1500.  1501.  1502.  1503.  1504.  1600.  1601.  1602.  1603.  1604.]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [193]:
d=ps.shape(X, n_in, features_test)

In [209]:
d[1,1,:]

array([ 300.,  301.,  302.,  303.,  304.])

# Reshape to 3d tenser

In [34]:
# ensure the shaping makes sense the way all the variables are arranged
# reshape input to be 3D [samples(observations), timesteps (n_in), features]
train_X = ps.shape(train_X, n_in=n_in, features=features)
X_validation = ps.shape(X_validation, n_in=n_in, features=features)
test_X = ps.shape(test_X, n_in=n_in, features=features)

In [36]:
print(train_X.shape, X_validation.shape, test_X.shape)

(1051, 10, 165) (375, 10, 165) (76, 10, 165)


# Build LSTM Architecture

In [ ]:
# for debug on lstm package
import imp
imp.reload(lstm)

In [ ]:
model = lstm.build_model(train_X, 
                         timesteps=n_in, 
                         inlayer=int(train_X.shape[-1]*20),
                         hiddenlayers=[60], 
                         outlayer=n_out)

Compilation Time :  0.01562643051147461
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 3300)          45751200  
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 3300)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 60)                806640    
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 305       
Total params: 46,558,145
Trainable params: 46,558,145
Non-trainable params: 0
_________________________________________________________________


# Fit model

In [ ]:
# fit network and save to history
history = model.fit(train_X, train_y, 
                    epochs=2500, 
                    batch_size=256, 
                    validation_data=(X_validation, Y_validation), 
                    verbose=2, shuffle=False)

# plot history
visualize.plot_loss(history)

Train on 1051 samples, validate on 375 samples
Epoch 1/2500
 - 10s - loss: 7040.2209 - val_loss: 23809.8292
Epoch 2/2500
 - 2s - loss: 6762.8850 - val_loss: 23566.7680
Epoch 3/2500
 - 2s - loss: 6664.2264 - val_loss: 23437.8161
Epoch 4/2500
 - 2s - loss: 6603.0976 - val_loss: 23306.5789
Epoch 5/2500
 - 2s - loss: 6541.2658 - val_loss: 23213.3759
Epoch 6/2500
 - 2s - loss: 6494.3763 - val_loss: 23118.2105
Epoch 7/2500
 - 2s - loss: 6442.1989 - val_loss: 23009.8884
Epoch 8/2500
 - 2s - loss: 6390.4680 - val_loss: 22918.9473
Epoch 9/2500
 - 2s - loss: 6339.6284 - val_loss: 22830.0732
Epoch 10/2500
 - 2s - loss: 6299.4858 - val_loss: 22741.7783
Epoch 11/2500
 - 2s - loss: 6250.4374 - val_loss: 22654.1721
Epoch 12/2500
 - 2s - loss: 6209.1477 - val_loss: 22567.4020
Epoch 13/2500
 - 2s - loss: 6162.3399 - val_loss: 22481.2544
Epoch 14/2500
 - 2s - loss: 6125.2696 - val_loss: 22395.8215
Epoch 15/2500
 - 2s - loss: 6084.9641 - val_loss: 22311.2531
Epoch 16/2500
 - 2s - loss: 6036.3712 - val_lo

 - 2s - loss: 2639.0449 - val_loss: 14569.6732
Epoch 135/2500
 - 2s - loss: 2627.9149 - val_loss: 14519.4882
Epoch 136/2500
 - 2s - loss: 2613.7502 - val_loss: 14469.4688
Epoch 137/2500
 - 2s - loss: 2579.9327 - val_loss: 14419.6041
Epoch 138/2500
 - 2s - loss: 2567.2789 - val_loss: 14369.9252
Epoch 139/2500
 - 2s - loss: 2534.5956 - val_loss: 14320.4869
Epoch 140/2500
 - 2s - loss: 2541.1910 - val_loss: 14271.3323
Epoch 141/2500
 - 2s - loss: 2502.8723 - val_loss: 14222.4242
Epoch 142/2500
 - 2s - loss: 2482.7563 - val_loss: 14173.6215
Epoch 143/2500
 - 2s - loss: 2472.7777 - val_loss: 14124.9560
Epoch 144/2500
 - 2s - loss: 2458.4377 - val_loss: 14076.5817
Epoch 145/2500
 - 2s - loss: 2454.1960 - val_loss: 14028.3731
Epoch 146/2500
 - 2s - loss: 2434.5007 - val_loss: 13980.4732
Epoch 147/2500
 - 2s - loss: 2394.4575 - val_loss: 13932.7655
Epoch 148/2500
 - 2s - loss: 2372.4349 - val_loss: 13885.2105
Epoch 149/2500
 - 2s - loss: 2379.3470 - val_loss: 13837.9720
Epoch 150/2500
 - 2s - 

Epoch 267/2500
 - 2s - loss: 1098.7888 - val_loss: 9513.8694
Epoch 268/2500
 - 2s - loss: 1091.2033 - val_loss: 9486.5602
Epoch 269/2500
 - 2s - loss: 1083.4218 - val_loss: 9459.4097
Epoch 270/2500
 - 2s - loss: 1081.3494 - val_loss: 9432.3843
Epoch 271/2500
 - 2s - loss: 1069.2197 - val_loss: 9405.4362
Epoch 272/2500
 - 2s - loss: 1046.6094 - val_loss: 9378.6239
Epoch 273/2500
 - 2s - loss: 1069.4124 - val_loss: 9352.0001
Epoch 274/2500
 - 2s - loss: 1043.0745 - val_loss: 9325.5879
Epoch 275/2500
 - 2s - loss: 1034.3675 - val_loss: 9299.3118
Epoch 276/2500
 - 2s - loss: 1042.4570 - val_loss: 9273.1572
Epoch 277/2500
 - 2s - loss: 1025.6974 - val_loss: 9247.0974
Epoch 278/2500
 - 2s - loss: 1023.6827 - val_loss: 9221.2299
Epoch 279/2500
 - 2s - loss: 1029.3994 - val_loss: 9195.4734
Epoch 280/2500
 - 2s - loss: 1019.5997 - val_loss: 9169.7500
Epoch 281/2500
 - 2s - loss: 991.3647 - val_loss: 9144.1460
Epoch 282/2500
 - 2s - loss: 1011.9099 - val_loss: 9118.7295
Epoch 283/2500
 - 2s - lo

Epoch 404/2500
 - 2s - loss: 669.3227 - val_loss: 6849.4156
Epoch 405/2500
 - 2s - loss: 684.0307 - val_loss: 6836.5653
Epoch 406/2500
 - 2s - loss: 675.6389 - val_loss: 6823.8672
Epoch 407/2500
 - 2s - loss: 692.6610 - val_loss: 6811.3933
Epoch 408/2500
 - 2s - loss: 682.5558 - val_loss: 6798.9759
Epoch 409/2500
 - 2s - loss: 680.5115 - val_loss: 6786.5399
Epoch 410/2500
 - 2s - loss: 669.4491 - val_loss: 6774.2297
Epoch 411/2500
 - 2s - loss: 691.9968 - val_loss: 6762.0554
Epoch 412/2500
 - 2s - loss: 693.9497 - val_loss: 6749.8497
Epoch 413/2500
 - 2s - loss: 681.9439 - val_loss: 6737.6361
Epoch 414/2500
 - 2s - loss: 690.3127 - val_loss: 6725.4840
Epoch 415/2500
 - 2s - loss: 697.8705 - val_loss: 6713.5371
Epoch 416/2500
 - 2s - loss: 682.8805 - val_loss: 6701.5225
Epoch 417/2500
 - 2s - loss: 686.4094 - val_loss: 6689.5112
Epoch 418/2500
 - 2s - loss: 684.8747 - val_loss: 6677.5788
Epoch 419/2500
 - 2s - loss: 653.2492 - val_loss: 6665.8068
Epoch 420/2500
 - 2s - loss: 689.7125 - 

Epoch 541/2500
 - 2s - loss: 664.0354 - val_loss: 5693.6665
Epoch 542/2500
 - 2s - loss: 649.4357 - val_loss: 5688.6691
Epoch 543/2500
 - 2s - loss: 664.5467 - val_loss: 5683.6526
Epoch 544/2500
 - 2s - loss: 658.7715 - val_loss: 5678.7400
Epoch 545/2500
 - 2s - loss: 674.2455 - val_loss: 5673.9103
Epoch 546/2500
 - 2s - loss: 667.7914 - val_loss: 5669.0981
Epoch 547/2500
 - 2s - loss: 683.1031 - val_loss: 5664.3701
Epoch 548/2500
 - 2s - loss: 666.5400 - val_loss: 5659.5093
Epoch 549/2500
 - 2s - loss: 680.6002 - val_loss: 5654.7743
Epoch 550/2500
 - 2s - loss: 683.5581 - val_loss: 5649.9357
Epoch 551/2500
 - 2s - loss: 667.6110 - val_loss: 5645.2044
Epoch 552/2500
 - 2s - loss: 666.5795 - val_loss: 5640.5824
Epoch 553/2500
 - 2s - loss: 652.7853 - val_loss: 5635.9729
Epoch 554/2500
 - 2s - loss: 659.8669 - val_loss: 5631.5361
Epoch 555/2500
 - 2s - loss: 651.8539 - val_loss: 5627.1377
Epoch 556/2500
 - 2s - loss: 678.9041 - val_loss: 5622.7578
Epoch 557/2500
 - 2s - loss: 673.2166 - 

Epoch 678/2500
 - 2s - loss: 662.9561 - val_loss: 5276.7279
Epoch 679/2500
 - 2s - loss: 654.8777 - val_loss: 5274.8579
Epoch 680/2500
 - 2s - loss: 702.4316 - val_loss: 5273.4572
Epoch 681/2500
 - 2s - loss: 687.7279 - val_loss: 5271.8944
Epoch 682/2500
 - 2s - loss: 684.4217 - val_loss: 5270.2564
Epoch 683/2500
 - 2s - loss: 687.9910 - val_loss: 5268.5989
Epoch 684/2500
 - 2s - loss: 671.7641 - val_loss: 5266.7500
Epoch 685/2500
 - 2s - loss: 698.8049 - val_loss: 5265.1523
Epoch 686/2500
 - 2s - loss: 686.0633 - val_loss: 5263.4845
Epoch 687/2500
 - 2s - loss: 703.6906 - val_loss: 5261.8816
Epoch 688/2500
 - 2s - loss: 688.7771 - val_loss: 5260.1380
Epoch 689/2500
 - 2s - loss: 692.9956 - val_loss: 5258.4350
Epoch 690/2500
 - 2s - loss: 684.4629 - val_loss: 5256.7969
Epoch 691/2500
 - 2s - loss: 693.3405 - val_loss: 5255.2273
Epoch 692/2500
 - 2s - loss: 685.4277 - val_loss: 5253.8021
Epoch 693/2500
 - 2s - loss: 684.7014 - val_loss: 5252.4754
Epoch 694/2500
 - 2s - loss: 683.4285 - 

Epoch 815/2500
 - 2s - loss: 685.9689 - val_loss: 5144.1271
Epoch 816/2500
 - 2s - loss: 697.2301 - val_loss: 5143.7041
Epoch 817/2500
 - 2s - loss: 708.8963 - val_loss: 5143.3899
Epoch 818/2500
 - 2s - loss: 689.0835 - val_loss: 5142.9418
Epoch 819/2500
 - 2s - loss: 708.0307 - val_loss: 5142.4634
Epoch 820/2500
 - 2s - loss: 696.1171 - val_loss: 5141.8449
Epoch 821/2500
 - 2s - loss: 698.0596 - val_loss: 5141.3681
Epoch 822/2500
 - 2s - loss: 701.2777 - val_loss: 5140.7906
Epoch 823/2500
 - 2s - loss: 690.3102 - val_loss: 5140.0856
Epoch 824/2500
 - 2s - loss: 705.5970 - val_loss: 5139.3971
Epoch 825/2500
 - 2s - loss: 700.4079 - val_loss: 5138.7205
Epoch 826/2500
 - 2s - loss: 692.6113 - val_loss: 5138.1083
Epoch 827/2500
 - 2s - loss: 698.7886 - val_loss: 5137.7719
Epoch 828/2500
 - 2s - loss: 683.1425 - val_loss: 5137.4215
Epoch 829/2500
 - 2s - loss: 708.1465 - val_loss: 5137.4135
Epoch 830/2500
 - 2s - loss: 683.5398 - val_loss: 5137.1181
Epoch 831/2500
 - 2s - loss: 681.2589 - 

Epoch 952/2500
 - 2s - loss: 694.2149 - val_loss: 5117.7188
Epoch 953/2500
 - 2s - loss: 691.9239 - val_loss: 5117.4716
Epoch 954/2500
 - 2s - loss: 720.1229 - val_loss: 5117.3443
Epoch 955/2500
 - 2s - loss: 746.0353 - val_loss: 5117.3527
Epoch 956/2500
 - 2s - loss: 700.9483 - val_loss: 5217.8088
Epoch 957/2500
 - 2s - loss: 666.0325 - val_loss: 5218.9825
Epoch 958/2500
 - 2s - loss: 695.5916 - val_loss: 5203.0976
Epoch 959/2500
 - 2s - loss: 672.4536 - val_loss: 5183.2630
Epoch 960/2500
 - 2s - loss: 658.5060 - val_loss: 5158.3285
Epoch 961/2500
 - 2s - loss: 681.1014 - val_loss: 5136.0071
Epoch 962/2500
 - 2s - loss: 657.4234 - val_loss: 5123.0220
Epoch 963/2500
 - 2s - loss: 659.3388 - val_loss: 5118.2156
Epoch 964/2500
 - 2s - loss: 675.0247 - val_loss: 5116.6069
Epoch 965/2500
 - 2s - loss: 656.3996 - val_loss: 5117.7078
Epoch 966/2500
 - 2s - loss: 658.9618 - val_loss: 5118.3095
Epoch 967/2500
 - 2s - loss: 653.7363 - val_loss: 5118.6805
Epoch 968/2500
 - 2s - loss: 673.0476 - 

In [ ]:
### KEEP THIS WORKING ####
yhat = predicts.predict_sequences_multiple(model, test_X, n_in, n_out)
visualize.plot_results_multiple(yhat, test_y[:,0], n_out, legend=False)
# visualize.plot_single(yhat, test_y, ticker, data_set_category='test')

In [ ]:
###### TODO METHOD####
#import predicts
## copy weights
#old_weights=model.get_weights()
#
#new_model = Sequential()
#new_model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]),
#          return_sequences=True, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(LSTM(10, activation='tanh'))
#new_model.add(Dropout(0.5))
#new_model.add(Dense(1))
#new_model.compile(loss='mae', optimizer='adam')
## fit network
#new_model.set_weights(old_weights)
#predicts.onlineForecast(new_model, test_X, test_y)
## # # ____ # # #